# Статистика, DS-поток
## Практическое задание 9


**Правила:**

* Дедлайн **9 ноября 16:00**. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить телеграм-боту `@miptstats_ad21_bot`.
* Прислать нужно ноутбук в формате `ipynb`, а также файл `utils.py` с вашим кодом.
* Решения, размещенные на каких-либо интернет-ресурсах не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него.
* Никакой код из данного задания при проверке запускаться не будет.

**Баллы за задание:**

* Задача 1 &mdash; 6 баллов;
* Задача 2 &mdash; 3 балла;
* Задача 3 &mdash; 2 балла;
* Задача 4 &mdash; 20 баллов.

**Важность задач:**

* *высокая:* задачи 1-3;
* *средняя:* задача 4.

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pof

from statsmodels.sandbox.stats.multicomp import multipletests

np.set_printoptions(precision=3)
pd.set_option("precision", 3)

_____
## Задача 2.

Рассмотрим выборку $X = (X_{1}, ..., X_{n}) \sim \mathcal{N}(\theta, 1)$. Проверяются гипотезы $\mathsf{H}_0\colon\ \theta = 0\ \ vs.\ \ \mathsf{H}_1\colon\ \theta\ > 0$.

Выпишите равномерно наиболее мощный критерий, который получен в предыдущем домашнем задании.

РНМК: $S = \{T(x) \geq c_{\alpha}\},$ где $T(x) = \sqrt{n}(\overline{X} - \theta_0), \ c_{\alpha}: 1 - \Phi{c_{\alpha}} = \alpha,$ откуда $c_{\alpha} = $ `sps.norm.ppf`($1 - \alpha$).

На практике перед применением критерия часто проверяют его на корректность следующим образом.
1. Генерируется множество выборок в предположении справедливости $\mathsf{H}_0$. 
2. По каждой из выборок вычисляется p-value.
3. По всем p-value строится гистограмма, которая визуально проверяется на равномерность.

Проверим корректность нашего критерия. Конечно, у нас случай довольно простой, тут и так "все понятно", но стоит отработать навык проверки.

Проведите эксперимент на $10^6$ выборок размера 100 и постройте гистограмму.

Пусть реализация статистики будет $t.$ Тогда p-value в нашем случае будет равно: `sps.norm.sf`($t$). <br>
Заметим, что распределение критерия статистики неодинаковое на $\Theta_0 = (-\infty, 0],$ так как если $X_i \sim \mathcal{N}(\theta, 1)$, то $T(X) \sim \mathcal{N}(\theta - \theta_0, 1)$ поэтому, вообще говоря, по замечанию из лекции стоит взять в качестве p-value $\sup_{\theta \in \Theta_0}p(\theta).$ Однако, легко заметить, что этот супремум достигается при $\theta = \theta_0 = 0,$ поэтому вместо супремума берем $p(\theta_0).$

In [6]:
sample_size = 100  # размер выборки
sample_count = 10**6  # количество выборок
theta_0 = 0

def build_histogram_from_theta(theta=0):
    samples = sps.norm(loc=theta, scale=1).rvs(size=(sample_size, sample_count))  # множество выборок
    Z_stats = np.sqrt(sample_size) * (np.average(samples, axis=0) - theta_0)  # статистика для каждой из выборок
    theta_grid = np.linspace(-10, 0, 101)
    pval_arr = np.array([np.zeros(sample_count)])
    pvalues = sps.norm.sf(Z_stats)

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=pvalues, xbins=dict(start=0,end=1,size=0.02)))
    pof.plot(fig, filename=f'uniform_{theta * 10 + 2}.html', auto_open=False)
    

build_histogram_from_theta()

Как известно, критерий не меняется при рассмотрении сложной основной гипотезы $\mathsf{H}_0\colon\ \theta \leqslant 0\ \ vs.\ \ \mathsf{H}_1\colon\ \theta\ > 0$.

Повторите эксперимент для истинного $\theta=-0.1$.

In [7]:
theta = -0.1
build_histogram_from_theta(theta=theta)

*Замечание.* Подобный эффект все равно не дает оснований говорить об уверенности $\mathsf{H}_0$, поскольку критерий может быть недостаточно мощным.

Проверьте, как ведет себя p-value для альтернативы, рассмотрев истинную $\theta=0.1$.

In [8]:
theta = 0.1
build_histogram_from_theta(theta=theta)

Чему соответствует доля случаев, в которых p-value меньше 0.05?

Если p-value меньше 0.05, то $H_0$ отклоняется, значит, доля случаев, в которых p-value меньше 0.05, соответствует доле отвергнутых гипотез от всех гипотез.

**Вывод:** при $\theta = 0$ наблюдается гистограмма, примерно соответствующая равномерному распределению. Однако при $\theta = -0.1 \in \Theta_0$ мы видим гистограмму, вообще не соответствующую равномерному распределению. Я думаю, это связано с тем, что распределение критерия статистики неодинаковое на $\Theta_0 = (-\infty, 0],$ поэтому не применимо утверждение с лекции о равномерности. При $\theta = 0.1 \notin \Theta_0$ мы видим гистограмму, вообще не соответствующую равномерному распределению, что логично, так как $\theta \notin \Theta_0,$ а значит, для него изначально не применимо утверждение с лекции о равномерности.